MAPBIOMAS

- Dados mapbiomas no GEE: https://developers.google.com/earth-engine/datasets/publisher/mapbiomas-public
- Dados MapBiomas: https://brasil.mapbiomas.org/mapbiomas-cobertura-10m/
- GitHub do MapBiomas: https://github.com/mapbiomas-brazil/user-toolkit
- Legenda Mapbiomas: https://brasil.mapbiomas.org/codigos-de-legenda/ e
https://github.com/mapbiomas-brazil/user-toolkit/tree/master/legend-colors

# Inicializando GEE

In [ ]:
# importa bibliotecas
import ee, geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import time

# inicializando pelo GEEMAP
geemap.ee_initialize(project='ee-enrique')

# Mapas do Mapbiomas

Carregando paleta de cores do MapBiomas

In [ ]:
# parâmetros de visualização
param_vis = {'min': 0,
             'max': 75,
             'palette': ['ffffff', '32a65e', '32a65e', '1f8d49', '7dc975',  # 0-4
                         '04381d', '026975', '000000', '000000', '7a6c00',  # 5-9
                         'ad975a', '519799', 'd6bc74', 'd89f5c', 'FFFFB2',  # 10-14
                         'edde8e', '000000', '000000', 'f5b3c8', 'C27BA0',  # 15-19
                         'db7093', 'ffefc3', 'db4d4f', 'ffa07a', 'd4271e',  # 20-24
                         'db4d4f', '0000FF', '000000', '000000', 'ffaa5f',  # 25-29
                         '9c0027', '091077', 'fc8114', '2532e4', '93dfe6',  # 30-34
                         '9065d0', 'd082de', '000000', '000000', 'f5b3c8',  # 35-39
                         'c71585', 'f54ca9', 'cca0d4', 'dbd26b', '807a40',  # 40-44
                         'e04cfa', 'd68fe2', '9932cc', 'e6ccff', '02d659',  # 45-49
                         'ad5100', '000000', '000000', '000000', '000000',  # 50-54
                         '000000', '000000', 'CC66FF', 'FF6666', '006400',  # 55-59
                         '8d9e8b', 'f5d5d5', 'ff69b4', 'ebf8b5', '000000',  # 60-64
                         '000000', '91ff36', '7dc975', 'e97a7a', '0fffe3',  # 65-69
                         '000000', '000000', '000000', '000000', '000000',  # 70-74
                         'c12100'  # 75
                          ]
              }

# legenda personalizada do mapbiomas
legenda_mapbiomas = {'3 - Formação Florestal': '1f8d49',
                     '4 - Formação Savânica': '7dc975',
                     '5 - Mangue': '04381d',
                     '6 - Floresta Alagável': '026975',
                     '9 - Floresta Plantada': '7a6c00',
                     '11 - Área Úmida': '519799',
                     '12 - Formação Campestre': 'd6bc74',
                     '13 - Outras Formações': 'd89f5c',
                     '15 - Pastagem': 'edde8e',
                     '18 - Agricultura': 'f5b3c8',
                     '20 - Cana-de-açúcar': 'db7093',
                     '21 - Mosaico de Usos': 'ffefc3',
                     '22 - Área não Vegetada': 'db4d4f',
                     '23 - Praia/Duna': 'ffa07a',
                     '24 - Área Urbana': 'd4271e',
                     '25 - Outras Áreas': 'db4d4f',
                     '29 - Afloramento Rochoso': 'ffaa5f',
                     '30 - Mineração': '9c0027',
                     '31 - Aquicultura': '091077',
                     '32 - Apicum': 'fc8114',
                     '33 - Rios/Lagos/Oceano': '2532e4',
                     '34 - Glaciar': '93dfe6',
                     '39 - Soja': 'f5b3c8',
                     '40 - Arroz': 'c71585',
                     '41 - Outras Lavouras': 'f54ca9',
                     '46 - Café': 'd68fe2',
                     '47 - Citros': '9932cc',
                     '48 - Outras Perenes': 'e6ccff',
                     '49 - Vegetação em Dunas': '02d659',
                     '50 - Vegetação em Dunas Herbácea': 'ad5100',
                     '61 - Salina': 'f5d5d5',
                     '62 - Algodão': 'ff69b4',
                     '69 - Recifes de Coral': '0fffe3',
                     '75 - Usina Solar': 'c12100'}

# extrai listas para a função add_legend
legenda_keys = list(legenda_mapbiomas.keys())
legenda_colors = ['#' + cor for cor in legenda_mapbiomas.values()]

# mostra os dados de nomes e cores
print(legenda_keys)
print(legenda_colors)

Carregando dados

In [ ]:
mapbiomas = ee.ImageCollection('projects/mapbiomas-public/assets/brazil/lulc/v1')
mapbiomas

In [ ]:
mapbiomas.first().bandNames().getInfo()

Plotando mapas em camadas

In [ ]:
# região
regiao = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
regiao = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM1_NAME', 'Minas Gerais'))
regiao = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Itajuba'))

# carrega dataset
mapbiomas = ee.ImageCollection('projects/mapbiomas-public/assets/brazil/lulc/v1') \
              .filter(ee.Filter.eq('collection_id', 10.0)) \
              .filter(ee.Filter.eq('version', 'v1'))

# seleciona 2020 e 2024
mapa_1990 = mapbiomas.filter(ee.Filter.eq('year', 1990)).first().clip(regiao)
mapa_2024 = mapbiomas.filter(ee.Filter.eq('year', 2024)).first().clip(regiao)

# cria a moldura do mapa
Map = geemap.Map()

# centraliza o mapa
Map.centerObject(regiao, 12)

# layers
Map.addLayer(mapa_1990, param_vis, '1990')
Map.addLayer(mapa_2024, param_vis, '2024')

# legenda
Map.add_legend(title="MapBiomas v10 - Classes",
               legend_dict=legenda_mapbiomas,
               position='bottomright',
               draggable=True)

# exibe na tela
Map

Plotando com Split map

In [ ]:
# região
regiao = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
regiao = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM1_NAME', 'Minas Gerais'))
regiao = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Itajuba'))

# carrega dataset
mapbiomas = ee.ImageCollection('projects/mapbiomas-public/assets/brazil/lulc/v1') \
              .filter(ee.Filter.eq('collection_id', 10.0)) \
              .filter(ee.Filter.eq('version', 'v1'))

# seleciona 2020 e 2024
mapa_1990 = mapbiomas.filter(ee.Filter.eq('year', 1990)).first().clip(regiao)
mapa_2024 = mapbiomas.filter(ee.Filter.eq('year', 2024)).first().clip(regiao)

# cria a moldura do mapa
Map = geemap.Map()

# centraliza o mapa
Map.centerObject(regiao, 12)

# split map
Map.split_map(left_layer=geemap.ee_tile_layer(mapa_1990, param_vis, 'Mapbiomas_2016'),
              right_layer=geemap.ee_tile_layer(mapa_2024, param_vis, 'Mapbiomas_2022'))

# exibe na tela
Map

Plotando com dois paineis

In [ ]:
# região
regiao = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
regiao = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM1_NAME', 'Minas Gerais'))
regiao = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Itajuba'))

# carrega dataset
mapbiomas = ee.ImageCollection('projects/mapbiomas-public/assets/brazil/lulc/v1') \
              .filter(ee.Filter.eq('collection_id', 10.0)) \
              .filter(ee.Filter.eq('version', 'v1'))

# seleciona 2020 e 2024
mapa_1990 = mapbiomas.filter(ee.Filter.eq('year', 1990)).first().clip(regiao)
mapa_2024 = mapbiomas.filter(ee.Filter.eq('year', 2024)).first().clip(regiao)

# monta painel com 4 mapas
geemap.linked_maps(rows = 1,
                   cols = 2,
                   height = "600px",
                   center = [-22.42, -45.4],
                   zoom = 12,
                   ee_objects = [mapa_1990, mapa_2024],
                   vis_params = [param_vis, param_vis],
                   labels = ['1990', '2024'],
                   label_position = "topright")

Salvando imagem no drive

In [ ]:
# salvando imagem tif
geemap.ee_export_image(mapa_2024,
                       filename='mapbiomas_itajuba_2024.tif',
                       scale=30,
                       region=regiao.geometry(),
                       file_per_band=False)

Instala e importa biblioteca para ler arquivo TIF

In [ ]:
# instala e importa biblioteca para ler arquivo TIF
!pip install rioxarray -q
import rioxarray

In [ ]:
# leitura do arquivo de relevo
ds = rioxarray.open_rasterio('/content/mapbiomas_itajuba_2024.tif')
ds

In [ ]:
# plota figura de um ano
ds[0,:,:].plot()

# Calculando a área: `geemap.image_area_by_group`

Use `geemap.image_area_by_group()` quando:
✅ Projeto rápido/prototipagem

✅ Não precisa de customização

✅ Prefere simplicidade sobre controle

✅ Trabalhando com MapBiomas/classificações padrão

✅ Quer resultados formatados rapidamente

In [ ]:
#============================================================================#
#                         CARREGA OS DADOS
#============================================================================#
# define a região
regiao = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Itajuba'))

# carrega os dados
mapbiomas = ee.ImageCollection('projects/mapbiomas-public/assets/brazil/lulc/v1') \
              .filter(ee.Filter.eq('collection_id', 10.0)) \
              .filter(ee.Filter.eq('version', 'v1')) \
              .map(lambda img: img.clip(regiao).copyProperties(img, ['system:time_start', 'year']))

# seleciona 2024
mapa_2024 = mapbiomas.filter(ee.Filter.eq('year', 2024)).first()

#============================================================================#
#                         CÁLCULO DA ÁREA
#============================================================================#
print("📊 Calculando área das classes...")
print("=" * 60)

# calcula área de cada classe em km²
resultado = geemap.image_area_by_group(mapa_2024,
                                       region=regiao,
                                       scale=30,
                                       denominator=1e6,
                                       decimal_places=4,
                                       verbose=True)

# Extrai as áreas, que estão na coluna 'area' do DataFrame resultado
areas_dict = resultado['area']

#============================================================================#
#                         CRIAÇÃO DA TABELA
#============================================================================#
# Cria lista para o DataFrame
dados_tabela = []

# Processa cada classe encontrada
for classe_num_str, area_km2 in areas_dict.items():

    # Converte chave para número
    classe_num = int(classe_num_str) if isinstance(classe_num_str, str) else classe_num_str

    # Encontra o nome da classe na legenda
    nome_classe = "Classe não identificada"

    for chave_legenda in legenda_mapbiomas.keys():
        if chave_legenda.startswith(f"{classe_num} -"):
            nome_classe = chave_legenda
            break

    # Adiciona à lista
    dados_tabela.append({'Código': classe_num,
                         'Classe': nome_classe.replace(f"{classe_num} - ", ""),
                         'Área (km²)': round(area_km2, 4),
                         'Área (ha)': round(area_km2 * 100, 2)  # Bônus: hectares
    })

# Cria DataFrame
df_areas = pd.DataFrame(dados_tabela)

# Ordena por área (do maior para o menor)
df_areas = df_areas.sort_values('Área (km²)', ascending=False).reset_index(drop=True)

#============================================================================#
#                              MOSTRA TABELA
#============================================================================#
print("\n" + "=" * 80)
print("📋 TABELA DE ÁREA POR CLASSE - ITAJUBÁ (2024)")
print("=" * 80)

# Exibe tabela formatada
print(df_areas.to_string(index=False, formatters={'Área (km²)': lambda x: f"{x:,.4f}",
                                                  'Área (ha)': lambda x: f"{x:,.2f}"}))
#============================================================================#
#                          ESTATÍSTICAS RESUMIDAS
#============================================================================#
print("\n" + "=" * 80)
print("📈 ESTATÍSTICAS RESUMIDAS")
print("=" * 80)

area_total = df_areas['Área (km²)'].sum()
num_classes = len(df_areas)

print(f"• Total de classes encontradas: {num_classes}")
print(f"• Área total mapeada: {area_total:,.2f} km²")
print(f"• Área total em hectares: {area_total * 100:,.0f} ha")

# Top 5 maiores classes
print(f"\n🏆 TOP 5 CLASSES POR ÁREA:")
top5 = df_areas.head(5)
for i, row in top5.iterrows():
    percentual = (row['Área (km²)'] / area_total) * 100
    print(f"  {i+1}. {row['Classe']}: {row['Área (km²)']:,.2f} km² ({percentual:.1f}%)")

# Calculando a área: `ee.Image.pixelArea()`

Use `ee.Image.pixelArea()` quando:
✅ Projeto de produção/robusto

✅ Precisa de máximo controle/performance

✅ Vai reutilizar o código em diferentes contextos

✅ Precisa customizar escala/geometria/reducer

✅ Quer entender exatamente o que está acontecendo

✅ Vai integrar com outros cálculos complexos

In [ ]:
#============================================================================#
#                         CARREGA OS DADOS
#============================================================================#
# define a região
regiao = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Itajuba'))

# carrega os dados
mapbiomas = ee.ImageCollection('projects/mapbiomas-public/assets/brazil/lulc/v1') \
              .filter(ee.Filter.eq('collection_id', 10.0)) \
              .filter(ee.Filter.eq('version', 'v1')) \
              .map(lambda img: img.clip(regiao).copyProperties(img, ['system:time_start', 'year']))

# seleciona 2024
mapa_2024 = mapbiomas.filter(ee.Filter.eq('year', 2024)).first()

#============================================================================#
#                         CÁLCULO DA ÁREA USANDO pixelArea()
#============================================================================#
print("=" * 80)
print("📊 Calculando área das classes usando ee.Image.pixelArea()...")
print("=" * 80)

# Obtém valores únicos das classes
valores_unicos = mapa_2024.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                                        geometry=regiao,
                                        scale=30,
                                        maxPixels=1e9).get('classification').getInfo()

# Converte para dicionário (se necessário)
if isinstance(valores_unicos, str):
    valores_unicos = eval(valores_unicos)

print(f"🔍 Encontradas {len(valores_unicos)} classes únicas")

# Cria dicionário para armazenar áreas
areas_dict = {}

# Calcula área para cada classe
for classe_str, num_pixels in valores_unicos.items():
    classe = int(classe_str)

    # Cria máscara para a classe específica (1 para pixels da classe, 0 para outros)
    mascara_classe = mapa_2024.eq(classe)

    # Calcula área em m²: pixelArea() retorna área por pixel em m²
    # Multiplica pela máscara para obter área apenas dos pixels da classe
    area_imagem = ee.Image.pixelArea().multiply(mascara_classe)

    # Soma a área total para essa classe
    area_total = area_imagem.reduceRegion(reducer=ee.Reducer.sum(),
                                          geometry=regiao,
                                          scale=30,
                                          maxPixels=1e9, bestEffort=True)

    # Obtém valor em m² e converte para km²
    area_m2 = area_total.getInfo().get('area', 0)
    area_km2 = area_m2 / 1e6  # Converte m² para km²

    areas_dict[classe] = area_km2

    # Mostra progresso
    print(f"  Classe {classe:2d}: {int(num_pixels):6d} pixels → {area_km2:8.4f} km²")

#============================================================================#
#                         CRIAÇÃO DA TABELA
#============================================================================#
# Cria lista para o DataFrame
dados_tabela = []

# Processa cada classe encontrada
for classe_num, area_km2 in areas_dict.items():

    # Encontra o nome da classe na legenda
    nome_classe = "Classe não identificada"

    for chave_legenda in legenda_mapbiomas.keys():
        if chave_legenda.startswith(f"{classe_num} -"):
            nome_classe = chave_legenda
            break

    # Adiciona à lista
    dados_tabela.append({'Código': classe_num,
                         'Classe': nome_classe.replace(f"{classe_num} - ", ""),
                         'Área (km²)': round(area_km2, 4),
                         'Área (ha)': round(area_km2 * 100, 2),  # Bônus: hectares
                         'Pixels': valores_unicos.get(str(classe_num), 0)})

# Cria DataFrame
df_areas = pd.DataFrame(dados_tabela)

# Ordena por área (do maior para o menor)
df_areas = df_areas.sort_values('Área (km²)', ascending=False).reset_index(drop=True)

#============================================================================#
#                              MOSTRA TABELA
#============================================================================#
print("\n" + "=" * 80)
print("📋 TABELA DE ÁREA POR CLASSE - ITAJUBÁ (2024) - Método pixelArea()")
print("=" * 80)

# Exibe tabela formatada
print(df_areas[['Código', 'Classe', 'Área (km²)', 'Área (ha)']].to_string(
    index=False,
    formatters={'Área (km²)': lambda x: f"{x:,.4f}",
                'Área (ha)': lambda x: f"{x:,.2f}"}))

#============================================================================#
#                          ESTATÍSTICAS RESUMIDAS
#============================================================================#
print("\n" + "=" * 80)
print("📈 ESTATÍSTICAS RESUMIDAS")
print("=" * 80)

area_total = df_areas['Área (km²)'].sum()
num_classes = len(df_areas)
total_pixels = df_areas['Pixels'].sum()

print(f"• Total de classes encontradas: {num_classes}")
print(f"• Total de pixels processados: {total_pixels:,}")
print(f"• Área total mapeada: {area_total:,.2f} km²")
print(f"• Área total em hectares: {area_total * 100:,.0f} ha")

# Verificação da área por pixel (deve ser ~900 m² para resolução 30m)
if total_pixels > 0:
    area_media_por_pixel = (area_total * 1e6) / total_pixels  # m² por pixel
    print(f"• Área média por pixel: {area_media_por_pixel:.1f} m²")
    print(f"  → Esperado: 900 m² (30m × 30m)")

# Top 5 maiores classes
print(f"\n🏆 TOP 5 CLASSES POR ÁREA:")
top5 = df_areas.head(5)
for i, row in top5.iterrows():
    percentual = (row['Área (km²)'] / area_total) * 100
    print(f"  {i+1}. {row['Classe']}: {row['Área (km²)']:,.2f} km² ({percentual:.1f}%)")

# Comparando 1990 e 2024

In [ ]:
#============================================================================#
#                         CARREGA OS DADOS
#============================================================================#
# define a região
regiao = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Itajuba'))

# carrega os dados
mapbiomas = ee.ImageCollection('projects/mapbiomas-public/assets/brazil/lulc/v1') \
              .filter(ee.Filter.eq('collection_id', 10.0)) \
              .filter(ee.Filter.eq('version', 'v1')) \
              .map(lambda img: img.clip(regiao).copyProperties(img, ['system:time_start', 'year']))

# seleciona 1990 e 2024
mapa_1990 = mapbiomas.filter(ee.Filter.eq('year', 1990)).first()
mapa_2024 = mapbiomas.filter(ee.Filter.eq('year', 2024)).first()

#============================================================================#
#                         CÁLCULO DA ÁREA - 1990
#============================================================================#
print("📊 Calculando área das classes para 1990...")
print("=" * 60)

# calcula área de cada classe em km² para 1990
resultado_1990 = geemap.image_area_by_group(mapa_1990,
                                           region=regiao,
                                           scale=30,
                                           denominator=1e6,
                                           decimal_places=4,
                                           verbose=True)

areas_dict_1990 = resultado_1990['area']

#============================================================================#
#                         CÁLCULO DA ÁREA - 2024
#============================================================================#
print("\n📊 Calculando área das classes para 2024...")
print("=" * 60)

# calcula área de cada classe em km² para 2024
resultado_2024 = geemap.image_area_by_group(mapa_2024,
                                           region=regiao,
                                           scale=30,
                                           denominator=1e6,
                                           decimal_places=4,
                                           verbose=True)

areas_dict_2024 = resultado_2024['area']

#============================================================================#
#                         UNIFORMIZAÇÃO DAS CLASSES
#============================================================================#
# Encontra todas as classes únicas entre os dois anos
todas_classes = set(list(areas_dict_1990.keys()) + list(areas_dict_2024.keys()))
print(f"\n🔍 Total de classes únicas encontradas: {len(todas_classes)}")

#============================================================================#
#                         CRIAÇÃO DA TABELA COMPARATIVA
#============================================================================#
# Cria lista para o DataFrame
dados_comparativos = []

# Processa cada classe
for classe_str in todas_classes:
    classe_num = int(classe_str) if isinstance(classe_str, str) else classe_str

    # Encontra o nome da classe na legenda
    nome_classe = "Classe não identificada"
    for chave_legenda in legenda_mapbiomas.keys():
        if chave_legenda.startswith(f"{classe_num} -"):

            nome_classe = chave_legenda
            break

    # Obtém áreas (ou 0 se não existir no ano)
    area_1990 = areas_dict_1990.get(classe_str, 0)
    area_2024 = areas_dict_2024.get(classe_str, 0)

    # Calcula mudanças
    mudanca_absoluta = area_2024 - area_1990

    # Calcula mudança percentual (evita divisão por zero)
    if area_1990 > 0:
        mudanca_percentual = (mudanca_absoluta / area_1990) * 100
    else:
        mudanca_percentual = 0 if area_2024 == 0 else 100  # Nova classe apareceu

    # Determina tendência
    if mudanca_absoluta > 0:
        tendencia = "📈 Aumento"
    elif mudanca_absoluta < 0:
        tendencia = "📉 Redução"
    else:
        tendencia = "⚖️ Estável"

    # Adiciona à lista
    dados_comparativos.append({'Código': classe_num,
                               'Classe': nome_classe.replace(f"{classe_num} - ", ""),
                               'Área 1990 (km²)': round(area_1990, 4),
                               'Área 2024 (km²)': round(area_2024, 4),
                               'Variação (km²)': round(mudanca_absoluta, 4),
                               'Variação (%)': round(mudanca_percentual, 2),
                               'Tendência': tendencia,
                               'Status': '✅ Presente' if area_1990 > 0 and area_2024 > 0 else '🆕 Nova' if area_1990 == 0 and area_2024 > 0 else
                               '❌ Desapareceu' if area_1990 > 0 and area_2024 == 0 else '❓' })

# Cria DataFrame
df_comparativo = pd.DataFrame(dados_comparativos)

# Ordena por variação absoluta (maiores mudanças primeiro)
df_comparativo = df_comparativo.sort_values('Variação (km²)',
                                           key=abs,
                                           ascending=False).reset_index(drop=True)

#============================================================================#
#                         TABELA PRINCIPAL COMPARATIVA
#============================================================================#
print("\n" + "=" * 100)
print("📊 TABELA COMPARATIVA - MAPBIOMAS 1990 vs 2024 - ITAJUBÁ")
print("=" * 100)

# Exibe tabela formatada
print(df_comparativo.to_string(
    index=False,
    formatters={'Área 1990 (km²)': lambda x: f"{x:,.4f}",
                'Área 2024 (km²)': lambda x: f"{x:,.4f}",
                'Variação (km²)': lambda x: f"{x:+,.4f}",
                'Variação (%)': lambda x: f"{x:+,.2f}%"}, max_colwidth=30))

#============================================================================#
#                         TABELA RESUMIDA (APENAS MUDANÇAS SIGNIFICATIVAS)
#============================================================================#
print("\n" + "=" * 100)
print("📈 RESUMO - MUDANÇAS SIGNIFICATIVAS (> 1 km²)")
print("=" * 100)

# Filtra apenas mudanças significativas
df_mudancas_significativas = df_comparativo[abs(df_comparativo['Variação (km²)']) > 1].copy()

if len(df_mudancas_significativas) > 0:
    print(df_mudancas_significativas[['Classe', 'Área 1990 (km²)', 'Área 2024 (km²)',
                                     'Variação (km²)', 'Variação (%)', 'Tendência']].to_string(
        index=False,
        formatters={'Área 1990 (km²)': lambda x: f"{x:,.2f}",
                    'Área 2024 (km²)': lambda x: f"{x:,.2f}",
                    'Variação (km²)': lambda x: f"{x:+,.2f}",
                    'Variação (%)': lambda x: f"{x:+,.1f}%"}))
else:
    print("Nenhuma mudança significativa (> 1 km²) encontrada.")

#============================================================================#
#                         ESTATÍSTICAS GERAIS
#============================================================================#
print("\n" + "=" * 100)
print("📊 ESTATÍSTICAS GERAIS DA MUDANÇA (1990-2024)")
print("=" * 100)

# Áreas totais
area_total_1990 = sum(areas_dict_1990.values)
area_total_2024 = sum(areas_dict_2024.values)
mudanca_total = area_total_2024 - area_total_1990

# Contagens
classes_1990 = len([a for a in areas_dict_1990.values if a > 0])
classes_2024 = len([a for a in areas_dict_2024.values if a > 0])
novas_classes = len([c for c in todas_classes if areas_dict_1990.get(str(c), 0) == 0 and areas_dict_2024.get(str(c), 0) > 0])
classes_perdidas = len([c for c in todas_classes if areas_dict_1990.get(str(c), 0) > 0 and areas_dict_2024.get(str(c), 0) == 0])

print(f"• Área total 1990: {area_total_1990:,.2f} km²")
print(f"• Área total 2024: {area_total_2024:,.2f} km²")
print(f"• Variação total: {mudanca_total:+,.2f} km² ({((mudanca_total/area_total_1990)*100):+.1f}%)")
print(f"• Classes em 1990: {classes_1990}")
print(f"• Classes em 2024: {classes_2024}")
print(f"• Novas classes: {novas_classes}")
print(f"• Classes desaparecidas: {classes_perdidas}")

# Análise de tendências
aumentos = df_comparativo[df_comparativo['Variação (km²)'] > 0]
reducoes = df_comparativo[df_comparativo['Variação (km²)'] < 0]
estaveis = df_comparativo[df_comparativo['Variação (km²)'] == 0]

print(f"\n• Classes em aumento: {len(aumentos)}")
print(f"• Classes em redução: {len(reducoes)}")
print(f"• Classes estáveis: {len(estaveis)}")

#============================================================================#
#                         TOP 5 AUMENTOS E REDUÇÕES
#============================================================================#
print("\n" + "=" * 100)
print("🏆 TOP 5 - MAIORES AUMENTOS")
print("=" * 100)

top5_aumentos = df_comparativo[df_comparativo['Variação (km²)'] > 0].nlargest(5, 'Variação (km²)')
for i, row in top5_aumentos.iterrows():
    print(f"{i+1}. {row['Classe']}: +{row['Variação (km²)']:,.2f} km² (+{row['Variação (%)']:,.1f}%)")

print("\n" + "=" * 100)
print("📉 TOP 5 - MAIORES REDUÇÕES")
print("=" * 100)

top5_reducoes = df_comparativo[df_comparativo['Variação (km²)'] < 0].nsmallest(5, 'Variação (km²)')
for i, row in top5_reducoes.iterrows():
    print(f"{i+1}. {row['Classe']}: {row['Variação (km²)']:+,.2f} km² ({row['Variação (%)']:+,.1f}%)")

#============================================================================#
#                         EXPORTAÇÃO DOS RESULTADOS
#============================================================================#
print("\n" + "=" * 100)
print("💾 EXPORTANDO RESULTADOS")
print("=" * 100)

# Exporta tabela completa
nome_csv = 'comparativo_mapbiomas_itajuba_1990_2024.csv'
df_comparativo.to_csv(nome_csv, index=False, encoding='utf-8-sig')
print(f"✅ Tabela completa exportada: '{nome_csv}'")

# Exporta resumo
resumo = pd.DataFrame({'Métrica': ['Área total 1990 (km²)', 'Área total 2024 (km²)', 'Variação total (km²)',
                                   'Variação total (%)', 'Classes 1990', 'Classes 2024',
                                   'Novas classes', 'Classes desaparecidas',
                                   'Classes em aumento', 'Classes em redução', 'Classes estáveis'],
                       'Valor': [round(area_total_1990, 2), round(area_total_2024, 2), round(mudanca_total, 2),
                                 round((mudanca_total/area_total_1990)*100, 1) if area_total_1990 > 0 else 0,
                                 classes_1990, classes_2024, novas_classes, classes_perdidas, len(aumentos), len(reducoes), len(estaveis)]})

nome_resumo = 'resumo_mudancas_itajuba_1990_2024.csv'
resumo.to_csv(nome_resumo, index=False, encoding='utf-8-sig')
print(f"✅ Resumo exportado: '{nome_resumo}'")

#============================================================================#
#                         VISUALIZAÇÃO DOS DADOS
#============================================================================#
print("\n" + "=" * 100)
print("📊 VISUALIZAÇÃO DOS DADOS")
print("=" * 100)

# Cria DataFrame para visualização
df_viz = pd.DataFrame({'Classe': df_comparativo['Classe'],
                       '1990': df_comparativo['Área 1990 (km²)'],
                       '2024': df_comparativo['Área 2024 (km²)'],
                       'Variação': df_comparativo['Variação (km²)']})

print("\nTop 10 classes por área em 2024:")
top10_2024 = df_viz.nlargest(10, '2024')[['Classe', '1990', '2024', 'Variação']]
print(top10_2024.to_string(index=False, formatters={'1990': lambda x: f"{x:,.2f}",
                                                    '2024': lambda x: f"{x:,.2f}",
                                                    'Variação': lambda x: f"{x:+,.2f}"}))

print("\n✅ Análise comparativa concluída!")

# Gráfico da evolução temporal da vegetação entre 1985 e 2024

## Processando cada ano

In [ ]:
%%time
#============================================================================#
#                         CARREGA OS DADOS - SÉRIE TEMPORAL
#============================================================================#
print("📊 INICIANDO EXTRACÇÃO DA SÉRIE TEMPORAL 1975-2024")
print("=" * 70)

# define a região
regiao = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Itajuba'))

# carrega todos os dados de 1975 a 2024
mapbiomas = ee.ImageCollection('projects/mapbiomas-public/assets/brazil/lulc/v1') \
              .filter(ee.Filter.eq('collection_id', 10.0)) \
              .filter(ee.Filter.eq('version', 'v1')) \
              .filter(ee.Filter.rangeContains('year', 1985, 2024)) \
              .map(lambda img: img.clip(regiao).copyProperties(img, ['system:time_start', 'year']))

# Obtém lista de anos disponíveis
anos_disponiveis = mapbiomas.aggregate_array('year').distinct().sort()
anos_lista = anos_disponiveis.getInfo()
print(f"📅 Anos disponíveis: {anos_lista}")
print(f"📊 Total de anos: {len(anos_lista)}")

#============================================================================#
#               CÁLCULO DA SÉRIE TEMPORAL PARA CLASSES SELECIONADAS
#============================================================================#
# Seleciona classes principais para análise
classes_principais = [3, 4, 12, 15, 18, 24, 33]  # Floresta, Savana, Pastagem, Agricultura, Urbano, Água
nomes_classes = {
    3: 'Floresta',
    4: 'Savana',
    12: 'Pastagem',
    15: 'Agricultura',
    18: 'Culturas',
    24: 'Urbano',
    33: 'Água'
}

cores_classes = {
    3: '#1f8d49',  # Verde floresta
    4: '#7dc975',  # Verde savana
    12: '#d6bc74', # Amarelo pastagem
    15: '#edde8e', # Amarelo claro agricultura
    18: '#f5b3c8', # Rosa agricultura
    24: '#d4271e', # Vermelho urbano
    33: '#2532e4'  # Azul água
}

print("\n🔍 Calculando série temporal para classes principais...")

# Dicionário para armazenar séries temporais
series_temporais = {classe: [] for classe in classes_principais}
anos_processados = []

# Processa cada ano
for ano in anos_lista:
    print(f"  Processando ano {ano}...")

    # Filtra imagem do ano
    imagem_ano = mapbiomas.filter(ee.Filter.eq('year', ano)).first()

    if imagem_ano:
        # Calcula área para cada classe principal
        for classe in classes_principais:
            # Cria máscara para a classe
            mascara_classe = imagem_ano.eq(classe)

            # Calcula área usando pixelArea()
            area_imagem = ee.Image.pixelArea().multiply(mascara_classe)

            # Soma a área total
            area_total = area_imagem.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=regiao,
                scale=30,
                maxPixels=1e9,
                bestEffort=True
            )

            # Obtém área em km²
            area_m2 = area_total.getInfo().get('area', 0)
            area_km2 = area_m2 / 1e6

            series_temporais[classe].append(area_km2)

        anos_processados.append(ano)

print(f"✅ Série temporal calculada para {len(anos_processados)} anos")

#============================================================================#
#                       CRIAÇÃO DO DATAFRAME
#============================================================================#
print("\n📋 Criando DataFrame com séries temporais...")

# Cria DataFrame
df_series = pd.DataFrame({
    'Ano': anos_processados
})

# Adiciona colunas para cada classe
for classe in classes_principais:
    df_series[nomes_classes[classe]] = series_temporais[classe]

# Calcula área total por ano
df_series['Área Total'] = df_series[[nomes_classes[c] for c in classes_principais]].sum(axis=1)

# Calcula porcentagens
for classe in classes_principais:
    nome = nomes_classes[classe]
    df_series[f'{nome} %'] = (df_series[nome] / df_series['Área Total']) * 100

In [ ]:
# mostra os dados
print(df_series.head())

## Plotando gráfico

In [ ]:
#============================================================================#
#               GRÁFICO COM EIXOS GÊMEOS PARA MELHOR VISUALIZAÇÃO
#============================================================================#
print("\n🎨 Criando gráfico com eixos gêmeos...")

# Configura estilo
plt.style.use('seaborn-v0_8-darkgrid')
fig, ax1 = plt.subplots(figsize=(16, 8))

# Divide as classes em dois grupos baseados na magnitude das áreas
# Analisa a magnitude média de cada classe
magnitudes = {}
for classe in classes_principais:
    nome = nomes_classes[classe]
    magnitudes[classe] = df_series[nome].mean()

# Ordena classes por magnitude
classes_ordenadas = sorted(classes_principais, key=lambda x: magnitudes[x], reverse=True)

# Separa em dois grupos: maiores e menores áreas
grupo_grande = classes_ordenadas[:3]  # 3 classes com maiores áreas
grupo_pequeno = classes_ordenadas[3:]  # Demais classes

print(f"📏 Grupo grande (eixo esquerdo): {[nomes_classes[c] for c in grupo_grande]}")
print(f"📐 Grupo pequeno (eixo direito): {[nomes_classes[c] for c in grupo_pequeno]}")

# ============== EIXO ESQUERDO (classes com maiores áreas) ==============
ax1.set_xlabel('Ano', fontsize=12, fontweight='bold')
ax1.set_ylabel('Área (km²) - Classes Grandes', fontsize=12, fontweight='bold', color='darkblue')

# Plota classes do grupo grande no eixo esquerdo
for classe in grupo_grande:
    nome = nomes_classes[classe]
    cor = cores_classes[classe]
    ax1.plot(df_series['Ano'], df_series[nome],
             label=f'{nome} (E)', color=cor, linewidth=3, marker='o', markersize=6)

ax1.tick_params(axis='y', labelcolor='darkblue')
ax1.set_xticks(df_series['Ano'][::2])
ax1.tick_params(axis='x', rotation=45)
ax1.set_xlim(df_series['Ano'].min() - 0.5, df_series['Ano'].max() + 0.5)
ax1.grid(True, alpha=0.3, linestyle='--')

# ============== EIXO DIREITO (classes com menores áreas) ==============
ax2 = ax1.twinx()
ax2.set_ylabel('Área (km²) - Classes Pequenas', fontsize=12, fontweight='bold', color='darkred')

# Plota classes do grupo pequeno no eixo direito
for classe in grupo_pequeno:
    nome = nomes_classes[classe]
    cor = cores_classes[classe]
    # Usa linha tracejada para diferenciar
    ax2.plot(df_series['Ano'], df_series[nome],
             label=f'{nome} (D)', color=cor, linewidth=2.5,
             linestyle='--', marker='s', markersize=5)

ax2.tick_params(axis='y', labelcolor='darkred')

# ============== CONFIGURAÇÕES GERAIS ==============
ax1.set_title('EVOLUÇÃO DO USO DO SOLO - ITAJUBÁ (1985-2024)\n' +
              'Eixo Esquerdo: Classes Grandes | Eixo Direito: Classes Pequenas',
              fontsize=14, fontweight='bold', pad=20)

# Combina legendas dos dois eixos
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2,
          loc='upper left', bbox_to_anchor=(1.02, 1),
          borderaxespad=0, fontsize=10, title='Classes\n(E=Esquerdo, D=Direito)')

# Adiciona linhas de referência verticais
anos_referencia = [1980, 1990, 2000, 2010, 2020]
for ano in anos_referencia:
    if ano in df_series['Ano'].values:
        ax1.axvline(x=ano, color='gray', linestyle=':', linewidth=0.8, alpha=0.5)

# Ajusta layout
plt.tight_layout(rect=[0, 0, 0.82, 1])
plt.show()